# Itinerary Generation

The purpose of this notebook is to generate the course site's itinerary page and subpages based on `Schedule.ipynb`

In [1]:
from datetime import date, timedelta
from helpers import calendar_table

In [2]:
SITE_DIR="../../../site"

In [3]:
%reload_ext autoreload
%run Schedule.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Date,Weekday,Plan
Week 0
Jan-10,Wed,FDOC
,,Day 00
Jan-12,Fri,Day 01
Week 1
Jan-15,Mon,University Holiday
Jan-17,Wed,Day 02
Jan-19,Fri,Day 03
Week 2
Jan-22,Mon,Day 04
Jan-24,Wed,Drop Period Ends
,,Day 05
Jan-26,Fri,Day 06
Week 3
Jan-29,Mon,Day 07
Jan-31,Wed,Day 08
Feb-02,Fri,Day 09
Week 4
Feb-05,Mon,Day 10
Feb-07,Wed,Day 11
Feb-09,Fri,Day 12
Week 5
Feb-12,Mon,University Holiday
Feb-13,Tue,University Holiday
Feb-14,Wed,Day 13
Feb-16,Fri,Day 14
Week 6
Feb-19,Mon,Day 15
Feb-21,Wed,Day 16
Feb-23,Fri,Day 17
Week 7
Feb-26,Mon,Day 18
Feb-28,Wed,Day 19
Mar-01,Fri,Day 20
Week 8
Mar-04,Mon,Day 21
Mar-06,Wed,Day 22
Mar-08,Fri,Widthdraw Period Ends
,,Day 23
Week 9
Mar-11,Mon,University Holiday
Mar-12,Tue,University Holiday
Mar-13,Wed,University Holiday
Mar-14,Thu,University Holiday
Mar-15,Fri,University Holiday
Week 10
Mar-18,Mon,Day 24
Mar-20,Wed,Day 25
Mar-22,Fri,Day 26
Week 11
Mar-25,Mon,Day 27
Mar-27,Wed,Day

We're going to curate what is in the full syllabus based on what we want to expose to students.

In [4]:
today = date.today() - timedelta(2)  # Hack to keep the past three days in range for now, need to rewrite for deadlines

def keep(plan):
    # Don't show "Class" or "Milestone"
    if plan.kind == Kind.MS or plan.kind == Kind.DK or plan.kind == Kind.NA:
        return False
    elif plan.kind == Kind.WD and plan.date - today > timedelta(6):
        # Filter holidays out in the far future, but keep nearby ones in for
        # positive vibes and unrelenting optimism.
        return False
    elif (plan.kind == Kind.CL or plan.kind == Kind.RV) and len(plan.links) == 0:
        return False
    elif plan.kind != Kind.QZ and plan.kind != Kind.EX and plan.kind != Kind.RD and plan.kind != Kind.FN and plan.date - today > timedelta(6):
        return False
    else:
        return True

for day in syllabus:
    remove = []
    syllabus[day] = [plan for plan in syllabus[day] if keep(plan)]

show_calendar(syllabus)

Date,Weekday,Plan
Week 0,,
Jan-10,Wed,Welcome to COMP110 - An Intro to the Class
,,Out: Syllabus and Support
,,Due: Syllabus and Support
Jan-12,Fri,Out: Hello World
,,An Introduction to Coding + VS Code Tutorial
,,Out: Visual Studio: Running Code
Week 1,,
Jan-15,Mon,Due: Visual Studio: Running Code
Jan-17,Wed,"Variables, Expressions, and User Input"


In [5]:
# Segment out future vs past
future = {}
past = {}
for day in syllabus:
    if day < today:
        past[day] = syllabus[day]
    else:
        future[day] = syllabus[day]

Similar to itinerary generation, we will auto-generate the 'On the Horizon' box. First, let's curate what is in the box:

In [6]:
from helpers import horizonize

horizon_syl = horizonize(syllabus_copy)
horizon_syl = [plan for plan in horizon_syl if plan.kind in {Kind.EX, Kind.QZ, Kind.RD, Kind.HW, Kind.FN, Kind.LS, Kind.PJ, Kind.CQ, Kind.VL}]

Stripping data to only near future quizzes and due dates

In [7]:
horizoninfo = []
today = date.today()

for plan in horizon_syl:
    if plan.date > (today - timedelta(1)) and plan.date < (today + timedelta(8)):
        horizoninfo.append(plan)

print(horizoninfo)

[<helpers.HorizonPlan object at 0x10686b190>, <helpers.HorizonPlan object at 0x10686b010>]


Working to add deadline in titles of assignments where assignment date != due date

In [8]:
import calendar
syllabus: dict[date, list[Plan]]
for day in syllabus:
    for plan in syllabus[day]: 
        if plan.deadline != plan.extension:
            newdate = plan.extension.strftime("%-m/%-d")
            date = plan.deadline.strftime("%-m/%-d")
            plan.title += f"<s><i>(Due {calendar.day_name[plan.deadline.weekday()]} {date})</i></s>" 
            plan.title += f" (Extended to {calendar.day_name[plan.extension.weekday()]} {newdate})" 
        elif (plan.deadline != None) and (plan.kind != Kind.QZ): 
            date = plan.deadline.strftime("%-m/%-d")
            plan.title += f" (Due {calendar.day_name[plan.deadline.weekday()]} {date})" 

And now (finally), we generate the itinerary and horizon box.

In [9]:
from jinja2 import Environment, FileSystemLoader, select_autoescape
from helpers import serialize, serialize_plan
import re
import json 
from datetime import date
env = Environment(
    loader=FileSystemLoader(SITE_DIR),
    autoescape=select_autoescape(["html", "xml"]),
    trim_blocks=True,
    lstrip_blocks=True
)
template = env.get_template("./index.jinja2")
with open(f"{SITE_DIR}/index.md", "w", encoding="utf8") as target:
    result = template.render(syllabus=syllabus, horizon_syl=horizon_syl, future=future, horizoninfo=horizoninfo, past=past, cache=date.today())
    # We strip the double spaces because pandoc can't handle HTML well :(
    stripped = re.sub(r' {2,}', '', result)
    target.write(stripped)
serializable_syllabus = serialize(syllabus) 
future_syllabus = serialize(future)  
past_syllabus = serialize(past)   
json_syllabus = json.dumps(serializable_syllabus, indent = 4)   
json_future = json.dumps(future_syllabus, indent = 4)
json_past = json.dumps(past_syllabus, indent = 4)
f = open(f"{SITE_DIR}/static/components/Itinerary/data.json", "w")
f.write(json_syllabus)
f.close()
f = open(f"{SITE_DIR}/static/components/Itinerary/future.json", "w")
f.write(json_future)
f.close()
f = open(f"{SITE_DIR}/static/components/Itinerary/past.json", "w")
f.write(json_past)
f.close()